In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras.utils import to_categorical

# ====== Load model kata ======
model_word = tf.keras.models.load_model("../models/sign_word_model.keras", compile=False)

# ====== Load label encoder kata ======
with open("../models/label_word_encoder.pkl", "rb") as f:
    le_word = pickle.load(f)
classes_word = list(le_word.classes_)
print(f"✅ Classes word: {classes_word}")

# ====== Load model huruf ======
model_letter = tf.keras.models.load_model("../models/sign_letter_model.keras", compile=False)

# ====== Load label encoder huruf ======
with open("../models/label_letter_encoder.pkl", "rb") as f:
    le_letter = pickle.load(f)
classes_letter = list(le_letter.classes_)
print(f"✅ Classes letter: {classes_letter}")

# ====== Inisialisasi Mediapipe ======
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

holistic = mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    enable_segmentation=False,
    refine_face_landmarks=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# ====== Fungsi ekstrak keypoints ======
def extract_keypoints(results):
    pose = np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]).flatten() \
        if results.pose_landmarks else np.zeros(33*3)

    lh = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark]).flatten() \
        if results.left_hand_landmarks else np.zeros(21*3)

    rh = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark]).flatten() \
        if results.right_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose, lh, rh])

# ====== Variabel untuk menyimpan urutan frame ======
sequence = []
seq_length = 30  # jumlah frame sebelum prediksi kata

# ====== Mulai kamera ======
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Kamera tidak terdeteksi.")
    exit()
print("✅ Kamera aktif. Tekan 'q' untuk keluar...")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Gagal membaca frame dari kamera.")
        break

    frame = cv2.flip(frame, 1)  # mirror
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(image_rgb)

    # Gambar landmark
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    # Ekstrak keypoints
    keypoints = extract_keypoints(results)
    sequence.append(keypoints)
    sequence = sequence[-seq_length:]

    # ====== Prediksi kata ======
    if len(sequence) == seq_length:
        X_input_word = np.expand_dims(sequence, axis=0)
        y_pred_word = model_word.predict(X_input_word, verbose=0)
        pred_word = classes_word[np.argmax(y_pred_word)]
        cv2.putText(frame, f"Word: {pred_word}", (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # ====== Prediksi huruf ======
    # Sesuaikan input dengan model huruf
    X_input_letter = np.expand_dims(keypoints, axis=0)  # shape (1, 225)
    y_pred_letter = model_letter.predict(X_input_letter, verbose=0)
    pred_letter_idx = np.argmax(y_pred_letter)
    pred_letter = le_letter.inverse_transform([pred_letter_idx])[0]

    cv2.putText(frame, f"Letter: {pred_letter}", (20, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Tampilkan frame
    cv2.imshow("Sign Translator", frame)

    # Tekan 'q' untuk keluar
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


✅ Classes word: ['aku', 'hai']
✅ Classes letter: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


I0000 00:00:1760424911.130979       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2025-10-14 13:55:11.227 python[3945:57703] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


✅ Kamera aktif. Tekan 'q' untuk keluar...


2025-10-14 13:55:13.693 python[3945:57703] +[IMKClient subclass]: chose IMKClient_Modern
2025-10-14 13:55:13.693 python[3945:57703] +[IMKInputSession subclass]: chose IMKInputSession_Modern


KeyboardInterrupt: 

: 